<a href="https://www.kaggle.com/code/nourberkdar/text-preprocessing-for-tweets?scriptVersionId=109023403" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentimental-analysis-for-tweets/sentiment_tweets3.csv


In [2]:
import re
import nltk
import spacy
import string

In [3]:
tweets_df=pd.read_csv('../input/sentimental-analysis-for-tweets/sentiment_tweets3.csv')
tweets_df.head()

,Index,message to examine,label (depression result)
0,106,just had a real good moment. i missssssssss hi...,0
1,217,is reading manga http://plurk.com/p/mzp1e,0
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,288,@lapcat Need to send 'em to my accountant tomo...,0
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0


In [4]:
import nltk
import re
from collections import Counter
from nltk.corpus import stopwords

nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk

True

In [5]:
tweets_df['word_count']=tweets_df['message to examine'].apply(lambda x:len(x.split(" ")))
tweets_df['word_count'].describe()

count    10314.000000
mean        16.066124
std         10.024623
min          1.000000
25%          9.000000
50%         14.000000
75%         21.000000
max         92.000000
Name: word_count, dtype: float64

In [6]:

tweets_df['char_count']=tweets_df['message to examine'].apply(len)
tweets_df['char_count'].describe()

count    10314.000000
mean        92.616153
std         64.670905
min          1.000000
25%         48.000000
50%         78.000000
75%        119.000000
max        874.000000
Name: char_count, dtype: float64

In [7]:
tweets_df

,Index,message to examine,label (depression result),word_count,char_count
0,106,just had a real good moment. i missssssssss hi...,0,12,57
1,217,is reading manga http://plurk.com/p/mzp1e,0,5,42
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0,5,84
3,288,@lapcat Need to send 'em to my accountant tomo...,0,23,135
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0,6,45
...,...,...,...,...,...
10309,802309,No Depression by G Herbo is my mood from now o...,1,20,101
10310,802310,What do you do when depression succumbs the br...,1,18,97
10311,802311,Ketamine Nasal Spray Shows Promise Against Dep...,1,10,96
10312,802312,dont mistake a bad day with depression! everyo...,1,10,57


## Vocabulary

Vocabulary: the set of unique vocabulary in the corpus
and optionally the count of each vocab

In [8]:
all_words=[word for tweet in tweets_df['message to examine'] for word in tweet.split(" ") ]

In [9]:
vocab=Counter(all_words)

In [10]:
print('Least Common')
vocab.most_common()[-10:]

Least Common


[('Bharat,u', 1),
 ('provoking', 1),
 ('sufferers', 1),
 ('sad;', 1),
 ('persistent', 1),
 ('nagging', 1),
 ('G', 1),
 ('Herbo', 1),
 ('succumbs', 1),
 ('https://cmun.it/4sjligkÂ\xa0', 1)]

In [11]:
def test_clean_fn(fn,sample):
    print(sample,fn(sample),sep='\n\n')

# Text Cleaning

In [12]:
sample=tweets_df.loc[0,"message to examine"]
sample

'just had a real good moment. i missssssssss him so much, '

### Stop Words

In [13]:
stop_words= stopwords.words("english")
stop_words[15:30]

['yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself']

In [14]:
stop_words=stopwords.words("english")
def remove_stop_words(text):
    return " ".join([word for word in text.split(" ") if word not in stop_words])

In [15]:
test_clean_fn(remove_stop_words,sample)

just had a real good moment. i missssssssss him so much, 

real good moment. missssssssss much, 


### Remove Hashtags

In [16]:
hashtag_re=re.compile(pattern='#[\w\d]+')
def remove_hashtag(text:str)->str:
    return hashtag_re.sub(repl="",string=text)


In [17]:
test_clean_fn(remove_hashtag,tweets_df.loc[6,"message to examine"])

@SilkCharm re: #nbn as someone already said, does fiber to the home mean we will all at least be regular now 

@SilkCharm re:  as someone already said, does fiber to the home mean we will all at least be regular now 


### Remove Mention


In [18]:
mention_re =re.compile('@\w+')
def remove_mention(text):
    return mention_re.sub(repl="",string=text)

test_clean_fn(remove_mention,tweets_df.loc[3,"message to examine"])

@lapcat Need to send 'em to my accountant tomorrow. Oddly, I wasn't even referring to my taxes. Those are supporting evidence, though. 

 Need to send 'em to my accountant tomorrow. Oddly, I wasn't even referring to my taxes. Those are supporting evidence, though. 


### Remove Punctuation Marks


In [19]:
punc_re = re.compile(r"""[!"#$%&\'()*+,-./:;<=>?@[\\\]^_`{|}~،؟…«“\":\"…”]""")

def remove_punctation(text:str)->str:
    return punc_re.sub(repl="",string=text)

In [20]:
test_clean_fn(remove_punctation,sample)

just had a real good moment. i missssssssss him so much, 

just had a real good moment i missssssssss him so much 


### Remove HTTP URLs

In [21]:
def remove_urls(text):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+))'," ",text)
test_clean_fn(remove_urls, tweets_df.loc[1,"message to examine"])

is reading manga  http://plurk.com/p/mzp1e

is reading manga   


### Remove Numbers

In [22]:
number_re=re.compile('\d+')
def remove_numbers(text):
    return number_re.sub(repl='',string=text)
    

### Remove Multiple Whitespace

In [23]:
multiple_space_re =re.compile('\s{2,}')
def remove_multiple_whitespace(text):
    return multiple_space_re.sub(repl=' ',string=text)

### Clean All

In [24]:
def clean_all(text):
    text=remove_urls(text)
    text=remove_hashtag(text)
    text=remove_mention(text)
    text=remove_punctation(text)
    text=remove_numbers(text)
    text=remove_stop_words(text)
    text=remove_multiple_whitespace(text)
    # remove space in beginning of text
    text = text.lower().strip(text)
    
    return text

In [25]:
tweets_df["clean_text"] = tweets_df["message to examine"].apply(clean_all)


In [26]:
tweets_df["clean_text"]

0             
1             
2             
3             
4        lookt
         ...  
10309        h
10310        w
10311        k
10312         
10313         
Name: clean_text, Length: 10314, dtype: object

# Text Normalisation

In [27]:
from nltk.stem.lancaster import LancasterStemmer
stemmer = nltk.LancasterStemmer()
stemmed_data = []
for sample in tweets_df["clean_text"]:
    words = sample.split(" ")
    stemmed_words = [stemmer.stem(word) for word in words]
    stemmed_data.append(" ".join(stemmed_words))



In [28]:
print(tweets_df['clean_text'][0],"\n\n",stemmed_data[0])
# print(stemmed_data[1])

In [29]:
tweets_df['clean_stemmed']=stemmed_data

In [30]:
tweets_df['clean_stemmed']

0             
1             
2             
3             
4        lookt
         ...  
10309        h
10310        w
10311        k
10312         
10313         
Name: clean_stemmed, Length: 10314, dtype: object

In [31]:
lm=nltk.WordNetLemmatizer()
Lemmatizer_data=[]
for sample in tweets_df['clean_text']:
    words=sample.split(' ')
    Lemmatizer_words=[lm.lemmatize(word) for word in words]
    Lemmatizer_data.append(" ".join(Lemmatizer_words))

In [32]:
print(tweets_df['clean_text'][0],"\n\n",Lemmatizer_data[0])


In [33]:
all_words=[word for tweet in Lemmatizer_data for word in tweet.split(" ")]
vocab=Counter(all_words)
counts=list(vocab.values())

In [34]:
vocab.most_common(n=10)

[('', 6391),
 ('i', 377),
 ('b', 319),
 ('j', 313),
 ('w', 296),
 ('m', 245),
 ('f', 229),
 ('h', 192),
 ('d', 186),
 ('c', 151)]

In [35]:
tweets_df.to_csv('data_tweets_clean',index=False)